In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests as rq

In [2]:
global overall, author, review_date, customer_review, aircarft, travller_type, cabin, route, date_flown, seat_comfort, cabin_service, food_bev, entertainment, ground_service, wifi, value_for_money, recommended

verified=[]
overall=[]
author=[]
review_date=[]
customer_review=[]
aircraft=[]
traveller_type=[]
cabin=[]
route=[]
date_flown=[]
seat_comfort=[]
cabin_service=[]
food_bev=[]
entertainment=[]
ground_service=[]
wifi=[]
value_for_money=[]
recommended=[]

In [3]:
def value_fetcher(soup):
    overall.append(np.nan)
    aircraft.append(np.nan)
    traveller_type.append(np.nan)
    cabin.append(np.nan)
    route.append(np.nan)
    date_flown.append(np.nan)
    seat_comfort.append(np.nan)
    cabin_service.append(np.nan)
    food_bev.append(np.nan)
    entertainment.append(np.nan)
    ground_service.append(np.nan)
    wifi.append(np.nan)
    value_for_money.append(np.nan)
    recommended.append(np.nan)

    for row in soup.find_all('tr'):
        header=row.find('td',class_='review-rating-header').text
        value=row.find('td',class_='review-value')
        if header=='Aircraft':
            aircraft[-1]=value.text
        elif header=='Type Of Traveller':
            traveller_type[-1]=value.text
        elif header=='Seat Type':
            cabin[-1]=value.text
        elif header=='Route':
            route[-1]=value.text
        elif header=='Date Flown':
            date_flown[-1]=value.text
        elif header=='Seat Comfort':
            seat_comfort[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Cabin Staff Service':
            cabin_service[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Food & Beverages':
            food_bev[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Inflight Entertainment':
            entertainment[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Ground Service':
            ground_service[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Wifi & Connectivity':
            wifi[-1]=len(row.find_all('span', class_='star fill'))
        elif header=='Value For Money':
            value_for_money[-1]=len(row.find_all('span',class_='star fill'))
        elif header=='Recommended':
            recommended[-1]=value.text

In [4]:
def fetcher(soup,tag,key,value):
    if soup.find(tag,{key:value}):
        return soup.find(tag,{key:value}).text
    return np.nan

In [5]:
for i in range(1,2):
    url=f'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/{i}/'
    page=rq.get(url).text
    soup=BeautifulSoup(page,'lxml')
    articles = soup.find_all('article',{'itemprop':'review'})
    for article in articles:
        author.append(fetcher(article, 'span', 'itemprop','author'))
        review_date.append(fetcher(article, 'time', 'itemprop', 'datePublished'))
        overall.append(fetcher(article,'span','itemprop','ratingValue'))
        div=article.find('div',class_='tc_mobile')
        content=(div.find('div',class_='text_content').text.split('|'))
        if len(content)==2:
            verification,review=content
        else:
            verification =np.nan
            review=content[0]
        verified.append(verification)
        customer_review.append(review)
        table=div.find('table',class_='review-ratings')
        value_fetcher(table)

In [6]:
data=pd.DataFrame({
    'status':verified,
    'author': author,
    'aircraft':aircraft,
    'review_date': review_date,
    'traveller_type':traveller_type,
    'cabin':cabin,
    'route':route,
    'date_flown':date_flown,
    'seat_comfort':seat_comfort,
    'cabin_service':cabin_service,
    'food_bev':food_bev,
    'entertainment':entertainment,
    'ground_service':ground_service,
    'wifi': wifi,
    'value_for_money':value_for_money,
    'recommended':recommended,
    'overall':overall[::2],
    'customer_review':customer_review})

In [7]:
data

,status,author,aircraft,review_date,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,wifi,value_for_money,recommended,overall,customer_review
0,Not Verified,\nMarlena Kovalska,NaN,12th October 2023,Couple Leisure,Economy Class,Athens to Warsaw,October 2023,5,5,5,5.0,5,5.0,4,yes,10,Very good and comfortable airline. There are ...
1,Not Verified,\nIoannis Kourkoulis,NaN,11th October 2023,Couple Leisure,Business Class,Santorini to Athens,September 2023,1,3,1,1.0,2,1.0,2,no,2,Terrible as Aegean always is over the last 2...
2,Not Verified,\nJ Wang,NaN,7th October 2023,Family Leisure,Economy Class,Heraklion to Santorini via Athen,October 2023,1,1,3,2.0,1,NaN,1,no,1,Heraklion to Athens only take 36 mins airtim...
3,✅ Trip Verified,\nChris Sifakis,NaN,5th October 2023,Couple Leisure,Economy Class,Athens to Heraklion,December 2022,3,3,1,1.0,3,NaN,1,no,1,After a very big delay in Vienna airport we...
4,✅ Trip Verified,\nE Garten,NaN,2nd October 2023,Couple Leisure,Economy Class,Athens to London Heathrow,October 2023,4,5,4,NaN,5,2.0,5,yes,9,"Great airline. On time, careful landing and ..."
5,✅ Trip Verified,\nWhitney Johnson,NaN,26th September 2023,Family Leisure,Economy Class,Mykonos to Athens,September 2023,2,1,1,NaN,1,NaN,1,no,1,On our connecting flight from Mykonos to At...
6,✅ Trip Verified,\nJ Boss,A320,25th September 2023,Business,Business Class,Venice to Athens,September 2023,1,2,1,NaN,1,NaN,1,no,1,I am gold on Aegean for the last ten years or...
7,✅ Trip Verified,\n6 reviews\n\n\n\nJ Murtar,NaN,25th September 2023,Family Leisure,Economy Class,Warsaw to Athens,August 2023,5,5,5,5.0,5,5.0,5,yes,10,It was my first experience with Aegean Airl...
8,Not Verified,\nPaola Zhamo,NaN,12th September 2023,Solo Leisure,Economy Class,Tirana to Seville via Athens,September 2023,1,1,1,1.0,1,1.0,1,no,1,Worst airline ever! I would never recommend ...
9,Not Verified,\nK Michael,NaN,8th September 2023,Couple Leisure,Economy Class,Athens to Chios,September 2023,1,1,1,1.0,1,1.0,1,no,1,The worst airline ever! They lost our luggage...
